In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../input/quora-question-pairs/train.csv.zip")

In [3]:
# Dataset Description

In [4]:
df.shape

In [5]:
df.head()

In [6]:
df.sample(10)

In [7]:
df.info()

In [8]:
df.isnull().sum()
df = df.dropna()

In [9]:
# Liter Dataset
#df = df.sample(200000)

In [10]:
questions_df = df[['question1','question2']]

In [11]:
!pip install contractions
!pip install session_info

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from string import punctuation
import re
import contractions

def tokenize(text):
    return word_tokenize(text)

def remove_stopwords(text):
    list_of_words = tokenize(text)
    _stopwords = set(stopwords.words("english"))
    filtered_list = [word + ' ' for word in list_of_words if word.casefold() not in _stopwords]
    return (''.join(filtered_list))[: -1] # to remove space at the end of string

def lemmatize(text):
    words = tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lem_words = [lemmatizer.lemmatize(word, pos='v')+' ' for word in words]
    return (''.join(lem_words))[: -1] # to remove space at the end of string

def preprocessing(text):
    text = re.sub('[0-9]+\.[0-9]+', "1", str(text)) # Remove float numbers
    constructed_txt = contractions.fix(text) # Remove shortened words such as "we'd" and return them to original words
    lem_txt = lemmatize(constructed_txt) # lemmatize words to get the original ones
    free_stopwords_txt = remove_stopwords(lem_txt) # remove stopwords
    text = ''.join([c for c in text if c not in punctuation]).lower()
    return text
    #print(text)

#preprocessing("youssef she's got 5.6 new dog this ?")
questions_df['question1'].apply(preprocessing)
questions_df['question2'].apply(preprocessing)

In [12]:
import session_info
session_info.show()

In [13]:
#print(questions_df['question1'])
#print(questions_df['question2'])
# ddf = pd.concat([questions_df['question1'],questions_df['question2']])
# print(ddf['question1'])
# print(ddf['question2'])


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
import scipy
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from sklearn.model_selection import train_test_split
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import losses 
from tensorflow.keras import optimizers
from tensorflow.keras import metrics


count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
new_df = pd.concat((questions_df['question1'],questions_df['question2']))
cv = count_vect.fit_transform(new_df)
trainq1_trans = count_vect.transform(questions_df['question1'].values.astype(str))
trainq2_trans = count_vect.transform(questions_df['question2'].values.astype(str))

labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

model = Sequential()
model.add(Dense(12, input_dim=X.get_shape()[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

print(X.get_shape())
print(X_train.get_shape())

model.compile(loss = 'mean_squared_error', optimizer = 'sgd', metrics = [metrics.categorical_accuracy])
model.fit(X_train, y_train, epochs=150, batch_size=10, verbose=0)

_, accuracy = model.evaluate(X_valid, y_valid)
print('Accuracy: %.2f' % (accuracy*100))

# xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(X_train, y_train) 
# xgb_prediction = xgb_model.predict(X_valid)

# from sklearn.metrics import f1_score, classification_report, accuracy_score

# print('training score:', f1_score(y_train, xgb_model.predict(X_train), average='macro'))
# print('validation score:', f1_score(y_valid, xgb_model.predict(X_valid), average='macro'))
# print(classification_report(y_valid, xgb_prediction))